In [16]:
from ebaysdk.finding import Connection as Finding
from ebaysdk.exception import ConnectionError
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime

In [2]:
completed_sales = []

for page in tqdm(range(1, 51)):
    try:
        api = Finding(config_file='./ebay.yaml')
        response = api.execute('findCompletedItems', {'keywords': 'Roman Coin',
                                 'paginationInput': {'pageNumber': page}})
        coin = response.dict()
        completed_sales.append(coin)

    except ConnectionError as e:
        print(e)


In [3]:
df = pd.DataFrame(completed_sales)

In [4]:
new = []
for item in df['searchResult']:
    new.append(item)

In [5]:
df2 = pd.DataFrame(new)

In [6]:
coins = []
for item in tqdm(df2['item']):
    for listing in item:
        coins.append(listing)

100%|██████████| 50/50 [00:00<00:00, 27101.99it/s]


In [7]:
roman = pd.DataFrame(coins)

In [8]:
primary_cat = []
for val in roman['primaryCategory']:
    primary_cat.append(val)

In [9]:
primary = pd.DataFrame(primary_cat)

In [10]:
prices = []

for price in roman['sellingStatus']:
    prices.append(price)
    
price_df = pd.DataFrame(prices)
price_df['currencyid_conv'] = [val['_currencyId'] for 
                               val in price_df['convertedCurrentPrice']]

price_df['value_conv'] = [val['value'] for 
                               val in price_df['convertedCurrentPrice']]

price_df['current_price'] = [val['value'] for 
                               val in price_df['currentPrice']]

price_df['current_currency'] = [val['_currencyId'] for 
                               val in price_df['currentPrice']]

In [11]:
price_df = price_df.drop(['convertedCurrentPrice', 'currentPrice'], axis=1)

In [12]:
roman_coins = pd.concat([roman, primary, price_df], axis=1)

In [17]:
#roman_coins.to_csv('./completed_sales_3_4')
file_name = 'completed_sales_' + re.sub('-', '_', str(datetime.datetime.now())[5:10])
roman_coins.to_csv(file_name)